# Optimization Project 1
### Rachana Aithal(ra39563)
### Jessica Lin (jl82983)
### Raghav Vaidya (rv25256)
### Sardar Muhammad Ahmad Ali (sa58382)

# 1)~3)

Assume that your company is deciding how to spend a marketing budget of $10M. You work in 
the marketing department as a data scientist and the chief marketing officer has asked you 
write a report recommending how to spread this budget among several marketing mediums. 
Your department has employed an outside consulting firm to estimate the return on investment 
(ROI) of each marketing medium under consideration. 


On top of these ROIs, your boss has decided to constrain your budget as follows:

a. The amount invested in print and TV should be no more than the amount spent on 
Facebook and Email. Surprisingly, email seems to be a great channel for reaching real 
people.

b. The total amount used in social media (Facebook, LinkedIn, Instagram, Snapchat, and 
Twitter) should be at least twice of SEO and AdWords.

c. For each platform, the amount invested should be no more than $3M.

Formulate the marketing budget allocation problem as a linear program. Use gurobi to find the 
optimal budget allocation.

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import quicksum

In [4]:
roi_data=pd.read_csv("ROI_data.csv")
variable_names = roi_data.columns.tolist()[1:]
variable_names

['Print',
 'TV',
 'SEO',
 'AdWords',
 'Facebook',
 'LinkedIn',
 'Instagram',
 'Snapchat',
 'Twitter',
 'Email']

In [5]:
roi_inital=list(roi_data.iloc[0][1:])

In [22]:
def calculate_roi(roi_list,budget):
    ojMod=gp.Model()
    #ojModX = ojMod.addMVar(10) # tell the model how many variables there are 
    ojModX=ojMod.addVars(len(variable_names),name=variable_names)
    #ojMod.setObjective(sum(roi * var for roi, var in zip(roi_list, ojModX)),sense=gp.GRB.MAXIMIZE)
    ojMod.setObjective(quicksum(roi_list[i]*ojModX[i] for i in range(len(variable_names))),gp.GRB.MAXIMIZE)
    conlist=[]
    conlist.append(ojMod.addConstr(ojModX[variable_names.index('Print')] + ojModX[variable_names.index('TV')] <= ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('Email')]))
    conlist.append(ojMod.addConstr(
                ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('LinkedIn')] + ojModX[variable_names.index('Instagram')] + \
                ojModX[variable_names.index('Snapchat')] + ojModX[variable_names.index('Twitter')] >= 2 * (ojModX[variable_names.index('SEO')] + \
                ojModX[variable_names.index('AdWords')])))
    for i in range(len(variable_names)):
        conlist.append(ojMod.addConstr(ojModX[i]<=3000000))
    conlist.append(ojMod.addConstr(quicksum(ojModX[i] for i in range(len(variable_names))) <= budget))
    ojMod.Params.OutputFlag = 0 
    ojMod.optimize() # solve the model
    return ojMod.objVal, ojMod.x


In [23]:
objval_inital,objX_inital=calculate_roi(roi_inital,10000000)
print("object val:",objval_inital)
print("variable values",objX_inital)

object val: 456000.0
variable values [0.0, 3000000.0, 0.0, 1000000.0, 0.0, 0.0, 3000000.0, 0.0, 0.0, 3000000.0]


## 4)~5)

Your boss is happy to see the promising results presented by the marketing department. 
However, your boss is also very concerned because your boss recalls being somewhat 
disappointed after following such recommendations in the past. To be cautious about the 
decision, your team has decided to get another opinion about the ROI data and rerun the 
analysis. The second consulting firm returns the estimates of the ROI data in the table below 
(also in the CSV file mentioned above). You are asked to compare the two optimal allocations 
from these two ROI estimates.

Are the allocations the same? Assuming the first ROI data is correct, if you were to use the 
second allocation (the allocation that assumed the second ROI data was correct) how much 
lower would the objective be relative to the optimal objective (the one that uses the first ROI 
data and the first allocation)? Assuming the second ROI data is correct, if you used the first 
allocation how much lower would the objective be relative to the optimal objective? Do you 
think the third constraint above, based on your boss’ experience, is useful?

In [24]:
roi_final=list(roi_data.iloc[1][1:])
objval_final,objX_final=calculate_roi(roi_final,10000000)
print("object val:",objval_final)
print("variable values:",objX_final)

object val: 456000.0
variable values: [3000000.0, 0.0, 0.0, 1000000.0, 3000000.0, 3000000.0, 0.0, 0.0, 0.0, 0.0]


Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)? Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective?

In [25]:
# Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)
alloc_2_roi_1=sum(roi * var for roi, var in zip(roi_inital, objX_final))
alloc_1_roi_1=sum(roi * var for roi, var in zip(roi_inital, objX_inital))
diff_obj1=alloc_1_roi_1-alloc_2_roi_1
print("The difference between the second allocation relative to the optimal objective: $",diff_obj1)

The difference between the second allocation relative to the optimal objective: $ 204000.0


In [26]:
# Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective?
alloc_1_roi_2=sum(roi * var for roi, var in zip(roi_final, objX_inital))
alloc_2_roi_2=sum(roi * var for roi, var in zip(roi_final, objX_final))
diff_obj2=alloc_2_roi_2-alloc_1_roi_2
print("The difference between the first allocation relative to the optimal objective: $",diff_obj2)

The difference between the first allocation relative to the optimal objective: $ 192000.0


Do you think the third constraint above, based on your boss’ experience, is useful?

In [30]:

ojMod=gp.Model()
#ojModX = ojMod.addMVar(10) # tell the model how many variables there are 
ojModX=ojMod.addVars(len(variable_names),name=variable_names)
#ojMod.setObjective(sum(roi * var for roi, var in zip(roi_list, ojModX)),sense=gp.GRB.MAXIMIZE)
ojMod.setObjective(quicksum(roi_inital[i]*ojModX[i] for i in range(len(variable_names))),gp.GRB.MAXIMIZE)
conlist=[]
conlist.append(ojMod.addConstr(ojModX[variable_names.index('Print')] + ojModX[variable_names.index('TV')] <= ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('Email')]))
conlist.append(ojMod.addConstr(
            ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('LinkedIn')] + ojModX[variable_names.index('Instagram')] + \
            ojModX[variable_names.index('Snapchat')] + ojModX[variable_names.index('Twitter')] >= 2 * (ojModX[variable_names.index('SEO')] + \
            ojModX[variable_names.index('AdWords')])))
conlist.append(ojMod.addConstr(quicksum(ojModX[i] for i in range(len(variable_names))) <= 10000000))
ojMod.Params.OutputFlag = 0 
ojMod.optimize() # solve the model
print("Without the 3M constraint per object object val: $",ojMod.objVal)
print("variable values:",ojMod.x)


Without the 3M constraint per object object val: $ 465000.0
variable values: [0.0, 5000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5000000.0]


Optimal Objective Value (ROI) increases after removing constraint_c, suggesting that this constraint_c based on the boss experience is not the best for Optimal Objective Value (ROI) but may consider other benefits.

## 6)

To explore this further perform some analysis of how your optimal allocation would change 
based on changes in the ROI data. Use the first ROI data as your starting point. By how much 
could each advertising medium’s ROI increase or decrease and still result in the same optimal 
allocation you found in step (3)?

In [32]:

ojMod=gp.Model()
#ojModX = ojMod.addMVar(10) # tell the model how many variables there are 
ojModX=ojMod.addVars(len(variable_names),name=variable_names)
#ojMod.setObjective(sum(roi * var for roi, var in zip(roi_list, ojModX)),sense=gp.GRB.MAXIMIZE)
ojMod.setObjective(quicksum(roi_inital[i]*ojModX[i] for i in range(len(variable_names))),gp.GRB.MAXIMIZE)
conlist=[]
conlist.append(ojMod.addConstr(ojModX[variable_names.index('Print')] + ojModX[variable_names.index('TV')] <= ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('Email')]))
conlist.append(ojMod.addConstr(
            ojModX[variable_names.index('Facebook')] + ojModX[variable_names.index('LinkedIn')] + ojModX[variable_names.index('Instagram')] + \
            ojModX[variable_names.index('Snapchat')] + ojModX[variable_names.index('Twitter')] >= 2 * (ojModX[variable_names.index('SEO')] + \
            ojModX[variable_names.index('AdWords')])))
conlist.append(ojMod.addConstr(quicksum(ojModX[i] for i in range(len(variable_names))) <= 10000000))
for i in range(len(variable_names)):
        conlist.append(ojMod.addConstr(ojModX[i]<=3000000))
ojMod.Params.OutputFlag = 0 
ojMod.optimize() # solve the model


In [34]:
sa = {
    'Platforms': list(roi_data.columns[1:]),
    'Min_roi': ojMod.SAObjLow,
    'Actual_roi': roi_inital,
    'Max_roi': ojMod.SAObjUp
}

# Create a DataFrame
sadf = pd.DataFrame(sa)
sadf["Min_roi"]=[max(0, value) for value in sadf["Min_roi"]]
sadf

,Platforms,Min_roi,Actual_roi,Max_roi
0,Print,0.000,0.031,0.049
1,TV,0.039,0.049,0.062
2,SEO,0.000,0.024,0.039
3,AdWords,0.033,0.039,0.046
4,Facebook,0.000,0.016,0.029
5,LinkedIn,0.000,0.024,0.039
6,Instagram,0.039,0.046,inf
7,Snapchat,0.000,0.026,0.039
8,Twitter,0.000,0.033,0.039
9,Email,0.029,0.044,inf


# 7)

Your boss has gained permission to reinvest half of the return. For example, if the marketing
obtains a 4% return in January, the budget of February will be $10M + $10M × 4% × 50% =
$10.2M. The monthly ROI for next year is given in an attached csv file. The three constraints
given by your boss are still in place for each month. What is the optimal allocation for each
month?

In [35]:
roi_mat = pd.read_csv('roi_mat.csv', index_col=0)
roi_mat=roi_mat/100

In [49]:
import calendar
budget_val=[]
budget_const=10000000
reinvestment=0
data = []
for i in range(0,len(roi_mat)):
    objval_inital,objX_inital=calculate_roi(roi_mat.iloc[i],budget)
    budget=budget_const+(reinvestment*budget)
    roi_month=objval_inital/sum(objX_inital)
    reinvestment=roi_month*0.5
    budget_val.append(budget)
    data.append(objX_inital)
print(budget_val)
months = list(calendar.month_name)[1:]
df = pd.DataFrame(data, index=months, columns=roi_data.columns[1:])
df=df/1000000


[10000000.0, 10186467.597206268, 10203389.80302422, 10203787.630012475, 10200168.820670081, 10205493.37871631, 10211913.770319, 10214141.755191302, 10218997.958646776, 10201360.847256895, 10185682.940491667, 10220788.058144368]


In [50]:
df['Total budget']=[budget / 1000000 for budget in budget_val]
df

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Total budget
January,3.000000,0.000000,0.0,1.406929,0.000000,0.0,2.813859,0.0,0.000000,3.000000,10.000000
February,3.000000,0.000000,0.0,2.333333,3.000000,0.0,0.000000,0.0,1.666667,0.000000,10.186468
March,0.000000,0.000000,0.0,3.000000,0.000000,3.0,1.186468,0.0,3.000000,0.000000,10.203390
April,0.000000,0.000000,0.0,3.000000,0.000000,3.0,3.000000,0.0,1.203390,0.000000,10.203788
May,1.203788,0.000000,0.0,0.000000,0.000000,0.0,3.000000,0.0,3.000000,3.000000,10.200169
June,3.000000,0.000000,0.0,0.000000,0.000000,0.0,3.000000,0.0,1.200169,3.000000,10.205493
July,0.000000,0.000000,0.0,3.000000,1.205493,0.0,3.000000,0.0,3.000000,0.000000,10.211914
August,2.711914,0.000000,0.0,1.500000,0.000000,0.0,0.000000,0.0,3.000000,3.000000,10.214142
September,0.607071,0.000000,0.0,3.000000,0.000000,3.0,0.000000,0.0,3.000000,0.607071,10.218998
October,0.000000,0.000000,0.0,3.000000,0.000000,3.0,3.000000,0.0,0.000000,1.218998,10.201361


In [51]:
print(f"The optimal budget allocation at the end of the year: {objval_inital/1000000:.6f}")

The optimal budget allocation at the end of the year: 0.431149


# 8)

A stable budget is defined as a monthly allocation such that for each platform the monthly
change in spend is no more than $1M. Is the allocation you found stable? If it isn’t, you do not
need to solve a new optimization model. Describe how we might model this?

In [53]:
# Calculate monthly budget change
monthly_budgetchange = df.diff()

# If monthly budget change abs > 1, then it is unstable
unstability = monthly_budgetchange.abs() > 1
unstability.replace({True: 'Unstable', False: '-'}, inplace=True)

# Unstability table
monthly_budgetchange.fillna(0, inplace=True)
monthly_budgetchange

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Total budget
January,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
February,0.000000,0.000000,0.0,0.926404,3.000000,0.0,-2.813859,0.0,1.666667,-3.000000,0.186468
March,-3.000000,0.000000,0.0,0.666667,-3.000000,3.0,1.186468,0.0,1.333333,0.000000,0.016922
April,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.813532,0.0,-1.796610,0.000000,0.000398
May,1.203788,0.000000,0.0,-3.000000,0.000000,-3.0,0.000000,0.0,1.796610,3.000000,-0.003619
June,1.796212,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,-1.799831,0.000000,0.005325
July,-3.000000,0.000000,0.0,3.000000,1.205493,0.0,0.000000,0.0,1.799831,-3.000000,0.006420
August,2.711914,0.000000,0.0,-1.500000,-1.205493,0.0,-3.000000,0.0,0.000000,3.000000,0.002228
September,-2.104843,0.000000,0.0,1.500000,0.000000,3.0,0.000000,0.0,0.000000,-2.392929,0.004856
October,-0.607071,0.000000,0.0,0.000000,0.000000,0.0,3.000000,0.0,-3.000000,0.611927,-0.017637


In [40]:
unstability

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Total budget
January,-,-,-,-,-,-,-,-,-,-,-
February,-,-,-,Unstable,Unstable,-,Unstable,-,Unstable,Unstable,-
March,Unstable,-,-,-,Unstable,Unstable,Unstable,-,Unstable,-,-
April,-,-,-,-,-,-,Unstable,-,Unstable,-,-
May,Unstable,-,-,Unstable,-,Unstable,-,-,Unstable,Unstable,-
June,Unstable,-,-,-,-,-,-,-,Unstable,-,-
July,Unstable,-,-,Unstable,Unstable,-,-,-,Unstable,Unstable,-
August,Unstable,-,-,Unstable,Unstable,-,Unstable,-,-,Unstable,-
September,Unstable,-,-,Unstable,-,Unstable,-,-,-,Unstable,-
October,Unstable,-,-,-,-,-,Unstable,-,Unstable,Unstable,-
